In [1]:
%cd ..

/home/juanhevia/IDIL


In [2]:
import idil_gym

Import error. Trying to rebuild mujoco_py.
running build_ext
building 'mujoco_py.cymj' extension
gcc -pthread -B /home/juanhevia/miniconda3/envs/aidil/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -I/home/juanhevia/miniconda3/envs/aidil/lib/python3.8/site-packages/mujoco_py -I/home/juanhevia/.mujoco/mujoco210/include -I/home/juanhevia/miniconda3/envs/aidil/lib/python3.8/site-packages/numpy/core/include -I/home/juanhevia/miniconda3/envs/aidil/lib/python3.8/site-packages/mujoco_py/vendor/egl -I/home/juanhevia/miniconda3/envs/aidil/include/python3.8 -c /home/juanhevia/miniconda3/envs/aidil/lib/python3.8/site-packages/mujoco_py/cymj.c -o /home/juanhevia/miniconda3/envs/aidil/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_38_linuxgpuextensionbuilder/temp.linux-x86_64-cpython-38/home/juanhevia/miniconda3/envs/aidil/lib/python3.8/site-packages/mujoco_py/cymj.o -fopenmp -w
gcc -pthread -B /home/juanhevia/miniconda3/envs/aidil/compiler_comp

In [3]:
# import os
# os.environ["LD_LIBRARY_PATH"] = os.environ.get("LD_LIBRARY_PATH", "") + ":/home/juanhevia/.mujoco/mujoco210/bin" + ":/usr/lib/nvidia"

import gymnasium
# import gym
import json 
import pickle
import numpy as np 
import pandas as pd 
import minari
# can we use the vector distance between the achieved and desired goal as a termination mark?
import matplotlib.pyplot as plt 

/home/juanhevia/miniconda3/envs/aidil/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Check proportion of macro goal sequences

In [13]:
# load train sequences
with open("notebooks/kitchen_franka_demos/FrankaKitchen-v0-mixed_260.pkl", "rb") as f:
    data = pickle.load(f)

In [14]:
from collections import Counter

rows = []

for traj in data["latents"]:
    _count = Counter(traj)
    # build JSON object as row
    rows.append(_count)

# convert to DataFrame
df = pd.DataFrame(rows)

In [15]:
df.isnull().any()

2    True
5    True
3    True
1    True
dtype: bool

In [35]:
unique_combs = []
for latent in data["latents"]:
    # compute the unique latents in the sequence without losing their order of appearance
    _unique = []
    for l in latent:
        if l not in _unique:
            _unique.append(l)
    
    unique_combs.append(tuple(_unique))

In [36]:
Counter(unique_combs)

Counter({(2, 5, 3): 68,
         (1, 5, 3): 86,
         (5, 3): 1,
         (1, 2, 5): 44,
         (1, 2, 3): 61})

In [37]:
sum(Counter(unique_combs).values())

260

## Split `pkl` files by macro goal sequence

In [40]:
available_seqs = list(Counter(unique_combs).keys())
# keep only 3-goals sequences
available_seqs = [seq for seq in available_seqs if len(seq) == 3]
available_seqs

[(2, 5, 3), (1, 5, 3), (1, 2, 5), (1, 2, 3)]

In [43]:
from collections import defaultdict

seqs_trajs = defaultdict(list)

# create a dict to hold trajectories indices for each sequence
for traj_idx, traj in enumerate(data["latents"]):
    # compute the unique latents in the sequence without losing their order of appearance
    _unique = []
    for l in traj:
        if l not in _unique:
            _unique.append(l)
            
    if tuple(_unique) in available_seqs:
        seqs_trajs[tuple(_unique)].append(traj_idx)
    
seqs_trajs.keys()

dict_keys([(2, 5, 3), (1, 5, 3), (1, 2, 5), (1, 2, 3)])

In [44]:
# check that lengths are the same
for seq, trajs in seqs_trajs.items():
    print(seq, " --> # of records = ", len(trajs))

(2, 5, 3)  --> # of records =  68
(1, 5, 3)  --> # of records =  86
(1, 2, 5)  --> # of records =  44
(1, 2, 3)  --> # of records =  61


In [45]:
# create new dictionaries to hold the sequences and the corresponding trajectories
split_data = {}

for seq, traj_indices in seqs_trajs.items():
    split_data[seq] = defaultdict(list)

    for key in data.keys():
        for idx in traj_indices:
            split_data[seq][key].append(data[key][idx])

In [48]:
# compute lengths of the sequences in the split data
for seq, data in split_data.items():
    print(seq, " --> keys : ", data.keys())
    print(seq, " --> # of records = ", len(data["latents"]))

    # ensure all keys in the data have equal length
    for key in data.keys():
        assert len(data[key]) == len(data["latents"])

(2, 5, 3)  --> keys :  dict_keys(['states', 'next_states', 'actions', 'rewards', 'latents', 'lengths', 'dones'])
(2, 5, 3)  --> # of records =  68
(1, 5, 3)  --> keys :  dict_keys(['states', 'next_states', 'actions', 'rewards', 'latents', 'lengths', 'dones'])
(1, 5, 3)  --> # of records =  86
(1, 2, 5)  --> keys :  dict_keys(['states', 'next_states', 'actions', 'rewards', 'latents', 'lengths', 'dones'])
(1, 2, 5)  --> # of records =  44
(1, 2, 3)  --> keys :  dict_keys(['states', 'next_states', 'actions', 'rewards', 'latents', 'lengths', 'dones'])
(1, 2, 3)  --> # of records =  61


In [49]:
# save the split data to different pickle files
for seq, data in split_data.items():
    seq_label = "_".join([str(s) for s in seq])
    with open(f"notebooks/kitchen_franka_demos/FrankaKitchen-v0-mixed_260_{seq_label}.pkl", "wb") as f:
        pickle.dump(data, f)

# Test env loading

In [6]:
env = gymnasium.make("CustomFrankaKitchen-v0")

TypeError: __init__() missing 1 required positional argument: 'env' was raised from the environment creator for CustomFrankaKitchen-v0 with kwargs ({})